Main islet class. Not much to change here between runs. The structure of self.locations and parameters to the constructor were changed (it is explained in greater detail below and in the main script [last code block])

In [ ]:
"""Script defining main islet class."""
import logging
# Setup logging
logger = logging.getLogger(__name__)

import math
import re

import numpy as np
from neuron import h

# Define global variables
# Size of cells
ALPHA_SIZE = DELTA_SIZE = 7
BETA_SIZE = 15.5
# Number of 'segments' to divide islet space into
SEGMENT_SIZE = 10


class Islet:
    def __init__(self, id: str, mechanism: str, islet_radius: int, simulation_update: int, cells: dict = None):
        """
        Initialize islet instance. Note that cells can be determined by the cells dict or the probabilities dict (which must contain the number of cells).

        Args:
            id (str): id of iselt.
            mechanism (str): mechanism to insert
            islet_radius (int): radius of islet.
            simulation_update (int): interval over which reset_values() will be called.
            cells (dict, optional): conditional dict (determines how many cells/probabilities/etc. See current example in simulate.ppy). Defaults to None.
        """
        logger.debug("Creating islet")
        
        # Set object variables from constructor parameters
        self._id = id
        self.mechanism = mechanism
        self.islet_radius = islet_radius
        self.simulation_update = simulation_update
        
        # Validate 'cells' parameter
        # TODO: add more validations (probabilities should add up to 100, should be less than 1, etc.)
        if 'type' not in cells or cells['type'] not in ['probabilistic', 'deterministic']:
            raise Exception("'type' needs to be a key in the cells object passed to the islet and should be one of 'probabilistic', 'deterministic'")
            
        # Set the following variables depending the 'type' of the 'cells' variable passed
        self._type = cells['type']
    
        # Probabilities are sent
        if cells['type'] == 'probabilistic':
            self.num_cells = cells['num_cells']
            self.prob_alpha = cells['A']
            self.prob_beta = cells['B']
            self.prob_delta = cells['D']
            self.num_betas = math.floor(self.num_cells * self.prob_beta)
            self.num_alphas = math.ceil(self.num_cells * self.prob_alpha)
            self.num_deltas = self.num_cells - self.num_betas - self.num_alphas
            
        # Number of cells are sent
        else:
            self.num_cells = cells['A'] + cells['B'] + cells['D']
            self.num_betas = cells['B']
            self.num_alphas = cells['A']
            self.num_deltas = cells['D']
        
        # Dict to store values to record
        self.cell_rec = dict()
        
        logger.debug("Islet created")

    def spatial_setup(self):
        """Setup each cell according to parameters passed to constructor with an appropriate spatial orientation."""
        logger.debug("Setting up potential coordinates for cells")
        
        # Number of segments to use
        nx, ny, nz = (SEGMENT_SIZE, SEGMENT_SIZE, SEGMENT_SIZE)
        
        # The below statements take a line from -radius to radius and chops it up into n segments
        x_coords = np.linspace(-self.islet_radius, self.islet_radius, nx)
        y_coords = np.linspace(-self.islet_radius, self.islet_radius, ny)
        z_coords = np.linspace(-self.islet_radius, self.islet_radius, nz)
        
        # Set locations for all cells
        self.locations = []
        count = 0
        while count < self.num_cells:
            
            # Select a random value for x, y, and z coordinates
            x,y,z = np.random.choice(x_coords, 1), np.random.choice(y_coords, 1), np.random.choice(z_coords, 1)
            
            # Check if the point is within the islet sphere
            if math.sqrt(x**2 + y**2 + z**2) < self.islet_radius:
                
                # Check if random coordinates chosen are unique so not two cells have the same location
                if any(location == [x[0],y[0],z[0]] for location in self.locations): 
                    continue
                else:
                    
                    # If location unique, append it to the locations list
                    self.locations.append({'x': x[0],'y': y[0], 'z': z[0]})
                    count+=1
                    
        logger.debug("Potential coordinates for islet setup")

    def populate_cells(self):
        """Create cell dictionary containing all the sections in the islet.""" 
        logger.debug("Populating cells")
        
        # Dictionary to populate
        self.cells = {}
       
       # Create the sections, which can be accessed by their name in the cells dictionary
        for i in range(self.num_alphas):
            self.cells["A_" + str(i)] = h.Section(name = "A_" + str(i))
        for i in range(self.num_betas):
            self.cells["B_" + str(i)] = h.Section(name = "B_" + str(i))
        for i in range(self.num_deltas):
            self.cells["D_" + str(i)] = h.Section(name = "D_" + str(i))

        # Insert one mechanism into each cell. In future make this like original setup
        # with config file and mechanism names for each cell type.
        for cell in self.cells:
            self.cells[cell].insert(self.mechanism)
            
        logger.debug("Cells populated")

    def set_cell_locations(self):
        """
        Separate the cells dictionary into lists for each cell type.
        This is accomplished by using the search criteria of if "A", "B", or "D" is in cell name.
        """
        logger.debug("Assigning coordinates to cells")
        
        # Get sections for all cell types
        alpha_sections = [val for key, val in self.cells.items() if "A" in key]
        beta_sections = [val for key, val in self.cells.items() if "B" in key]
        delta_sections = [val for key, val in self.cells.items() if "D" in key]
        
        
        count = 0
        logger.debug(f"Setting up:")
        logger.debug(f"{self.num_alphas} alpha cells")
        logger.debug(f"{self.num_betas} beta cells")
        logger.debug(f"{self.num_deltas} delta cells")
        
        for cell_idx in range(self.num_cells):
            
            while count < self.num_alphas:
                alpha_sections[cell_idx].pt3dclear()
                alpha_sections[cell_idx].pt3dadd(self.locations[cell_idx]['x'], self.locations[cell_idx]['y'], self.locations[cell_idx]['z'], ALPHA_SIZE)
                alpha_sections[cell_idx].pt3dadd(self.locations[cell_idx]['x'] + ALPHA_SIZE, self.locations[cell_idx]['y'], self.locations[cell_idx]['z'], ALPHA_SIZE)
                count+=1
            
            while self.num_alphas <= count < self.num_alphas + self.num_betas:
                beta_sections[cell_idx].pt3dclear()
                beta_sections[cell_idx].pt3dadd(self.locations[cell_idx]['x'], self.locations[cell_idx]['y'], self.locations[cell_idx]['z'], BETA_SIZE)
                beta_sections[cell_idx].pt3dadd(self.locations[cell_idx]['x'] + BETA_SIZE, self.locations[cell_idx]['y'], self.locations[cell_idx]['z'], BETA_SIZE)
                count+=1
            
            while self.num_alphas + self.num_betas <= count < self.num_cells:
                delta_sections[cell_idx].pt3dclear()
                delta_sections[cell_idx].pt3dadd(self.locations[cell_idx]['x'], self.locations[cell_idx]['y'], self.locations[cell_idx]['z'], DELTA_SIZE)
                delta_sections[cell_idx].pt3dadd(self.locations[cell_idx]['x'] + DELTA_SIZE, self.locations[cell_idx]['y'], self.locations[cell_idx]['z'], DELTA_SIZE)   
                count+=1    

        logger.debug("Coordinates assigned to cells")
    
    
    def record_values(self):
        """Create dictionary that will store values of all variables of interest present in all mechanisms."""
        logger.debug("Setting up recording dictionary")
        
        # Get all density mechanisms and variables using NEURON defined 'psection()'
        for cell in self.cells:
            self.cell_rec[cell] = dict()
            for mechanism in self.cells[cell].psection()['density_mechs']:
                for variable in self.cells[cell].psection()['density_mechs'][mechanism]:
                    
                    # Get density mechanism name
                    head = re.split("[0-9]", mechanism)[0]
                    
                    # Set name of variable to record
                    self.cell_rec[cell][str(head + '_' + variable)] = []

                    # Record variables of every mechanism in every segment
                    for k in self.cells[cell]:
                        mechRecord = getattr(k, '_ref_'+variable+'_'+mechanism)
                        self.cell_rec[cell][str(head + '_' + variable)].append(h.Vector().record(mechRecord))        

        logger.debug("Recording dictionary setup")
        
    def reset_values(self):
        """Update record variables to decrease memory usage"""
        logger.debug("Reseting values")
        for cell in self.cell_rec:
            for var in self.cell_rec[cell]:
                
                # Remove up to the last variable or the simulation_update size
                length_var = len(self.cell_rec[cell][var]) if var == 'Time' else len(self.cell_rec[cell][var][0].remove(0))
                for i in range(min(self.simulation_update, length_var)):
                    
                    # 'Time' variable has a different structure
                    if var == 'Time':
                        self.cell_rec[cell][var].remove(0)
                    else:
                        self.cell_rec[cell][var][0].remove(0)
                        
                logger.debug(f"Reset {var}")
                        
        logger.debug("Values reset")
    
    def __repr__(self):
        """String representation for debugging."""
        return "Islet '{id}' of type '{type}'".format(id=self._id, type=self._type)

This next script corresponds to 'Helper.py' in the scripts/ folder and contains multiple test cases in the create_distance_matrix function. These test cases, intuitively, allow us to modify distance/replicate the Watts model. 

Please note that the function listed at the bottom (dump_variables) was implemented to get around long simulation times, it may not be necessary for smaller simulations (An alternative is to set the SIMULATION_UPDATE variable in the 'simulation.py' script to larger than the size of the simulation, this will result in only one dump at the end of the simulation). 

In the case you want to exclude it, comment it out from the main script (code block after this one).

In the case you want to include the dump_variables function, keep in mind that if the size of the cells_rec dict exceeds the memory allocated to the session, the run will fail.

In [ ]:
"""Script containing functions used to perform calculations/simulations."""
import logging
# Setup logging
logger = logging.getLogger(__name__)

import math
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


def create_dist_matrices(islet_name: str):
    """
    This function first creates a matrix that contains the distance between each cell.
    Each row of this matrix corresponds to a cell.
    Say there are k alpha cells, l beta cells, and m delta cells.
    And let n=k+l+m be the total number of cells.
    The first k rows correspond to alpha cells.
    Rows k+1 to k+l correspond to beta cells.
    Rows k+l+1 to n correspond to delta cells.
    This matrix will then be subsetted into different matrices
    One for beta cells affecting alphas that will contain the rows corresponding to 
    beta cells and the columns corresponding to alpha cells (D_ba).
    One for delta cells affecting alphas that will contain the rows corresponding to 
    delta cells and the columns corresponding to alpha cells (D_da).
    One for alpha cells affecting betas that will contain the rows corresponding to 
    alpha cells and the columns corresponding to beta cells (D_ab).
    One for delta cells affecting betas that will contain the rows corresponding to 
    delta cells and the columns corresponding to beta cells (D_db).
    One for beta cells affecting deltas that will contain the rows corresponding to 
    beta cells and the columns corresponding to delta cells (D_bd).

    Args:
        islet_name (str): islet id.

    Returns:
        (dict): dictionary indexed by id of matrix to be returned (contains total dist_matrix and all subsets used in calculations).
    """
    logger.debug("Setting up distance matrix")
    
    # Create a numpy array containing the distance between each cell
    dist_matrix = np.array([math.dist((location_1['x'], location_1['y'], location_1['z']), (location_2['x'], location_2['y'], location_2['z'])) for location_1 in islet_name.locations for location_2 in islet_name.locations])
    
    # Reshape the array to be nxn where each row corresponds to a single cell and the 
    # distances between itself and all other cells
    dist_matrix = np.reshape(dist_matrix, (islet_name.num_cells, islet_name.num_cells))
    
    # TEST: use ordered list to test logic used to create the subsets below
    dist_matrix = np.arange(100)
    dist_matrix.shape = (10,10)
    logger.info("TEST CASE: using sorted list (np.arrange)")
    
    # TEST: use zero matrix to reproduce watts model
    dist_matrix = np.zeros((10,10))
    logger.info("TEST CASE: using zero matrix (reproduce watts model)")
    
    # TEST: use ones matrix to modify watts model via increasing distances between cells (all cells will be the same distance apart in this case)
    dist = 100
    dist_matrix = np.ones((10,10)) * dist
    logger.info("TEST CASE: using scalar multiple of ones matrix (module watts model with uniform distance)")
    
    # Create D_ba
    D_ba = dist_matrix[0:islet_name.num_alphas, islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas]
    # Create D_da
    D_da = dist_matrix[0:islet_name.num_alphas, islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells]
    # Create D_ab
    D_ab = dist_matrix[islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas, 0:islet_name.num_alphas]
    # Create D_db
    D_db = dist_matrix[islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas, islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells]
    # Create D_bd
    D_bd = dist_matrix[islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells, islet_name.num_alphas:islet_name.num_alphas+islet_name.num_betas]
    # Create D_ad
    D_ad = dist_matrix[islet_name.num_alphas+islet_name.num_betas:islet_name.num_cells, 0:islet_name.num_alphas]
    
    return {
        "distance_matrix": dist_matrix,
        "beta_affecting_alpha": D_ba,
        "beta_affecting_delta": D_bd,
        "delta_affecting_alpha": D_da,
        "delta_affecting_beta": D_db,
        "alpha_affecting_beta": D_ab,
        "alpha_affecting_delta": D_ad
        }


def calculate_secretion_rate_matrix(islet_name: str, matrices: dict):
    """
    Function to make column vectors of JIS, JGS, and JSS values.
    These vectors will need to be populated with new values at each time step

    Args:
        islet_name (str): islet id.
        matrices (dict): dictionary indexed by id of matrix to be returned (contains total dist_matrix and all subsets used in calculations).
    """
    # logger.debug("Calculate secretion rate matrix")
    
    # Create list of cell names by accessing keys in cell
    # dictionary
    cell_names = list(sorted(islet_name.cells.keys()))
    
    # Initialize column vectors
    JIS_col_matrix = []
    JGS_col_matrix = []
    JSS_col_matrix = []
    
    # Add JIS, JGS, JSS values to corresponding vector based on name
    for cell in cell_names:
        if "B" in cell:
            JIS_col_matrix.append(islet_name.cells[cell](0.5).one.JIS)
        if "A" in cell:
            JGS_col_matrix.append(islet_name.cells[cell](0.5).one.JGS)
        if "D" in cell:
            JSS_col_matrix.append(islet_name.cells[cell](0.5).one.JSS)
    
    # logger.debug("insulin from beta cell")
    # logger.debug(JIS_col_matrix)
    # logger.debug("glucagon from alpha cell")
    # logger.debug(JGS_col_matrix)
    # logger.debug("somatostatin from delta cell")
    # logger.debug(JSS_col_matrix)
    # input()
    
    # Calculate new secretion rates
    JGS_col_d = list(np.matmul(matrices["alpha_affecting_delta"], JGS_col_matrix))
    JGS_col_b = list(np.matmul(matrices["alpha_affecting_beta"], JGS_col_matrix))
    JIS_col_a = list(np.matmul(matrices["beta_affecting_alpha"], JIS_col_matrix))
    JIS_col_d = list(np.matmul(matrices["beta_affecting_delta"], JIS_col_matrix))
    JSS_col_a = list(np.matmul(matrices["delta_affecting_alpha"], JSS_col_matrix))
    JSS_col_b = list(np.matmul(matrices["delta_affecting_beta"], JSS_col_matrix))
    
    # logger.debug("insulin")
    # logger.debug(JIS_col_a)
    # logger.debug(JIS_col_d)
    # logger.debug("glucagon")
    # logger.debug(JGS_col_d)
    # logger.debug(JGS_col_b)
    # logger.debug("somatostatin")
    # logger.debug(JSS_col_a)
    # logger.debug(JSS_col_b)
    # input()
    
    # Set secretion rates
    for cell in sorted(islet_name.cells):
        if "A" in cell:
            islet_name.cells[cell](0.5).one.JIS = JIS_col_a[0] + islet_name.cells[cell](0.5).one.JIS
            JIS_col_a.pop(0)
            islet_name.cells[cell](0.5).one.JSS = JSS_col_a[0] + islet_name.cells[cell](0.5).one.JSS
            JSS_col_a.pop(0)
            # logger.debug(f"{cell} "set JSS {islet_name.cells[cell](0.5).one.JSS} JIS {islet_name.cells[cell](0.5).one.JIS}")
        elif "B" in cell:
            islet_name.cells[cell](0.5).one.JGS = JGS_col_b[0] + islet_name.cells[cell](0.5).one.JGS
            JGS_col_b.pop(0)
            islet_name.cells[cell](0.5).one.JSS = JSS_col_b[0] + islet_name.cells[cell](0.5).one.JSS
            JSS_col_b.pop(0)
            # logger.debug(f"{cell} set JGS {islet_name.cells[cell](0.5).one.JGS} JSS {islet_name.cells[cell](0.5).one.JSS}")
        elif "D" in cell:
            islet_name.cells[cell](0.5).one.JIS = JIS_col_d[0] + islet_name.cells[cell](0.5).one.JIS
            JIS_col_d.pop(0)
            islet_name.cells[cell](0.5).one.JGS = JGS_col_d[0] + islet_name.cells[cell](0.5).one.JGS
            JGS_col_d.pop(0)
            # logger.debug(f"{cell} set JIS {islet_name.cells[cell](0.5).one.JIS} JGS {islet_name.cells[cell](0.5).one.JGS}")
        

def visualize_parameter(cell_rec_dict: dict, vars: list, plot_path: str):
    """
    Simple function that allows us to use cells recording dictionary to visualize one parameter versus time.

    Args:
        cell_rec_dict (dict): dictionary indexed by variable name containing h.Vector() objects containing recorded variable values.
        vars (list): variables to plot.
        plot_path (str): path to which plot will be saved.
    """
    logger.debug("Visualizing parameter")
    
    Time = cell_rec_dict['Time']
    for var in vars:
        parameter = cell_rec_dict[var][0]
        fig = plt.figure()
        plt.plot(Time, parameter)
        plt.xlabel("Time (ms)")
        plt.ylabel(var)
        fig.savefig(f"{plot_path}_{var}.png")


def plot_parameters(cell_rec_dict: dict, vars: list, plot_path: str):
    """
    Function to compare plots from our simulations to those from the BAD model. Uses rec dictionary from the cell.

    Args:
        cell_rec_dict (dict): dictionary indexed by variable name containing h.Vector() objects containing recorded variable values.
        vars (list): variables to plot.
        plot_path (str): path to which plot will be saved.
    """
    # logger.debug("Plotting...")
    Time = cell_rec_dict['Time']
    fig, axes = plt.subplots(nrows = 3, sharex = True)
    fig.suptitle(f"{' '.join(vars.upper())}")
    for idx, var in enumerate(vars):
        axes[idx].plot(Time, cell_rec_dict[var][0])
        axes[0].set_ylabel(f"{var}")
    fig.savefig(plot_path)


def dump_variables(islet: Islet, temporary_path: str, step: int):
    """
    Function to dump variables to csv, clear memory and resume simulation.

    Args:
        islet (Islet): main islet object.
        temporary_path (str): path of temporary csv file.
        step (int): step number in simulation.
    """
    logger.debug("Dumping variables..")
    # Create pandas dataframe using dictionary of recorded values.
    final_dict = dict()
    for cell in islet.cell_rec:
        header_field = cell
        for var in islet.cell_rec[cell]:
            
            # Set time field
            if var == 'Time' and 'Time' not in final_dict:
                final_dict[var] = islet.cell_rec[cell][var]
            
            header_field += f"_{var}"
            
            # Note that list is stored at index 0 of the h.Vector() object
            final_dict[header_field] = islet.cell_rec[cell][var][0]
            
    # Create and write pandas dataframe
    df = pd.DataFrame({key: pd.Series(value) for key, value in final_dict.items()})
    logger.info(f"Dataframe size: {str(df.size)}")
    # input()
    if step == 0:
        
        # Remove the file if it exists
        logger.debug("Replacing temporary file..")
        os.system(f"rm {temporary_path}")
        df.to_csv(temporary_path, index=False)
    else:
        logger.debug("Appending to temporary file..")
        with open(temporary_path, 'a') as temporary_file:
            df.to_csv(temporary_file, index=False, header=False)
        logger.debug("Appended to temporary csv")
        
        # Reset variables
        islet.reset_values()
        logger.debug("Reset variables")


This next 'main' code block contains the code that actually runs the simulation. There are a couple test cases that allow us to modify how the islet is created. The islet can be created with a predeteremined number of cells or probabilities and a target number of cells for the islet.

See above note about commenting out the dump_variables() functions for shorter simulations.

Please note that verbosity of logs can be reduced be setting "level=logging.INFO".

In [ ]:
"""Main simulation script"""
import logging
import re

import coloredlogs

# Setup logging (needs to be done before any file imports)
# coloredlogs.install()
logging.basicConfig(
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("debug.log"),
        logging.StreamHandler()
    ],
    level=logging.DEBUG
)

import os
from timeit import default_timer as timer

import psutil
from neuron import h


# Load neuron?
h.load_file("stdrun.hoc")


# Global Variables (potentially modify before each run)
# Suffix of mechanism in mod file
MECHANISM = "one"
OUTPUT_FOLDER = "Test"
SIMULATION_TIME = 10
# Interval over which variables will be dumped/logs will print
SIMULATION_UPDATE = 40000
ISLET_ID = "one_islet"
ISLET_RADIUS = 1
TEMP_CSV = '.temp.csv'

Isolate test cases/islet creation

In [ ]:

# TEST: create islet based on number of cells per type
cells = {
    "type": "deterministic", 
    "A": 1, 
    "B": 1, 
    "D": 1
}
logging.info("TEST CASE: determinstic input")

# TEST: create islet based on probabilities of difference cell types and total number of cells
# cells = {
#     "type": "probabilistic", 
#     "A": 0.3, 
#     "B": 0.2, 
#     "D": 0.5, 
#     "num_cells": 10
# }
# logging.info("TEST CASE: probabilistic input")

# Create islet
test_islet = Islet(
    id=ISLET_ID, 
    mechanism=MECHANISM, 
    islet_radius=ISLET_RADIUS, 
    simulation_update=SIMULATION_UPDATE,
    cells=cells
)
test_islet.spatial_setup()
test_islet.populate_cells()
test_islet.record_values()
test_islet.set_cell_locations()
logging.info(f"{test_islet} created")

Set time variable

In [ ]:
# Set time variable per dictionary recording each cell
# TODO: modify, did out of convenience
for cell in test_islet.cell_rec:
    test_islet.cell_rec[cell]['Time'] =  h.Vector().record(h._ref_t)
    


Run initial block

In [ ]:
# Run initial block of mod file
h.finitialize()

Create distance matrix

In [ ]:
# Create distance matrix and subsets corresponding to effect of cell x on y (to be used in matrix multiplication)
matrices = create_dist_matrices(test_islet)

Run simulation

In [ ]:
# Keep track of real time elapsed since simulation started
real_start_time = timer()
for i in range(40 * SIMULATION_TIME):

        # Advance one time step
        h.fadvance()
        
        # Perform matrix math to calculate new secretion rates
        calculate_secretion_rate_matrix(test_islet, matrices)
        
        # Perform logging and variable dumps every SIMULATION_UPDATE timesteps (.025 * 10^-3 * 4 * 10^3 = .1 seconds if SIMULATION_UPDATE = 4 * 10^3)
        if i % SIMULATION_UPDATE == 0:
            
            # Simulation time elapsed in seconds
            simulation_time_elapsed = (0.025 * i) / 1000

            # Get memory usage statistics
            pid = os.getpid()
            python_process = psutil.Process(pid)
            memory_use_gb = python_process.memory_info()[0]/2.**30 
            memory_use_percent = psutil.virtual_memory().available * 100 / psutil.virtual_memory().total
            
            # Log
            logging.info(f"Simulation time: {simulation_time_elapsed}. Real time: {timer()-real_start_time}. Memory Usage: {memory_use_gb}GB / {memory_use_percent}%")
            
            # Dump variables at this interval          
            dump_variables(test_islet, TEMP_CSV, i)
           
# Dump variables at conclusion of simulation
dump_variables(test_islet, TEMP_CSV, -1, last=True)

Rewrite cell_rec variable

In [ ]:
# Read csv and reset the cell_rec variable
df = pd.read_csv(TEMP_CSV)
# Use columns to rewrite cell_rec object
for column in df.columns:
    match = re.search(r'((?:A|B|D)_[0-9])_(.*)', column)
    if match is not None:
        cell, var = list(match.groups())
        test_islet.cell_rec[cell][var] = df[column].to_list()

Plots

In [ ]:
      
# Setup plotting results folder
os.system(f"mkdir -p Plots/{OUTPUT_FOLDER}")
cell_plot_path = "Plots/{output_folder}/{cell_id}"

# Plot time series for the following variables for each cell
variables_to_plot = {
    "A": ["va", "ca", "G"],
    "B": ["vb", "c", "I"],
    "D": ["vd", "cd", "S"]
    }

# Plot each cell
for cell in test_islet.cell_rec:
    logging.info(f"Plotting cell: {cell}")
    
    # Note that 'cell[0]' will be one of 'A'/'B'/'D'
    # When not using dump_variables use the following
    plot_parameters(test_islet.cell_rec[cell], variables_to_plot[cell[0]], cell_plot_path.format(output_folder=OUTPUT_FOLDER, cell_id=cell), mechanism=MECHANISM)
    